In [2]:
from transformers import BertModel, BertTokenizerFast, BertTokenizer, BertConfig
from transformers import DistilBertModel, DistilBertTokenizerFast, DistilBertTokenizer, DistilBertConfig
import torch
from flask import Flask, jsonify, request
from torch import nn

In [3]:
bc = BertConfig(output_hidden_states=True)
b = BertModel(bc)
b = b.from_pretrained("bert-base-uncased")
bt = BertTokenizer.from_pretrained("bert-base-uncased")
btf = BertTokenizerFast.from_pretrained("bert-base-uncased")

dbc = DistilBertConfig(output_hidden_states=True)
db = DistilBertModel(dbc)
db.output_hidden_states = True
db = db.from_pretrained("distilbert-base-multilingual-cased")
dbt = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
dbtf = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased")


In [13]:
utterances = ["I'm trying to compute embeddings",'why is my results different?', 'there is an error in the result', 'there is a problem with the result', 'something unknown is not working right', 'my results are not what it is supposed to be', "i've got a problem with my results", "i'm not getting the results i expected", 'i have a problem, i expected something different', 'i have an unknown problem', 'i got the wrong results for some reason', 'i expected a different result', 'i can not get the right results']
len(utterances)

13

In [15]:
print(bt.tokenize(utterances[0]))
print(len(bt.tokenize(utterances[0])))
print(bt.encode(utterances[0]))
print(bt.encode(utterances[0], return_tensors="pt").size())
e = b(bt.encode(utterances[0], return_tensors="pt"))
print(len(e))
print(e[0].size())
# Sentence (last hidden) can be computed by summing /averaging the e[0]
print(e[1].size())

['i', "'", 'm', 'trying', 'to', 'compute', 'em', '##bed', '##ding', '##s']
10
[101, 1045, 1005, 1049, 2667, 2000, 24134, 7861, 8270, 4667, 2015, 102]
torch.Size([1, 12])
2
torch.Size([1, 12, 768])
torch.Size([1, 768])


In [19]:
i = bt.batch_encode_plus(utterances,pad_to_max_length=True, return_tensors="pt")
o, h = b(i["input_ids"],i["attention_mask"])
print(o.size())
print(h.size())

torch.Size([13, 12, 768])
torch.Size([13, 768])


In [28]:
avgpool = nn.AdaptiveAvgPool1d(300)
o = b(i["input_ids"], i["attention_mask"],i["attention_mask"])
print(o[0].size())
print(torch.sum(o[0],axis=1).size())
print(avgpool(torch.sum(o[0],axis=1).unsqueeze(1)).size())

torch.Size([12, 12, 768])
torch.Size([12, 768])
torch.Size([12, 1, 300])


In [9]:
ib = torch.tensor(bt.batch_encode_plus(utterances,pad_to_max_length=True, return_tensors="pt")['input_ids']).unsqueeze(0)
ibf = torch.tensor(btf.batch_encode_plus(utterances)['input_ids']).unsqueeze(0)

idb = torch.tensor(dbt.batch_encode_plus(utterances)['input_ids']).unsqueeze(0)
idbf = torch.tensor(dbtf.batch_encode_plus(utterances)['input_ids']).unsqueeze(0)

print(len(dbtf.encode(sentence)))

ValueError: expected sequence of length 8 at dim 1 (got 9)

In [23]:
ob = b(ib) 
obf = b(ibf) 

In [24]:
odb = db(idb)
odbf = db(idbf)

In [25]:
print(len(ob))
print(ob[0].size())
print(ob[1].size())
print(len(obf))
print(obf[0].size())
print(obf[1].size())

2
torch.Size([1, 18, 768])
torch.Size([1, 768])
2
torch.Size([1, 18, 768])
torch.Size([1, 768])


In [32]:
print(len(odb))
print(odb[0].size())
print(len(odbf))
torch.sum(odb[0],axis=1).size()


1
torch.Size([1, 18, 768])
1


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [33]:
jsonify(odb[0])

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.